In [1]:
import os
import pandas as pd
import tempfile
import numpy as np
from concurrent.futures import ProcessPoolExecutor
from functools import partial
import librosa 
from tqdm import tqdm
import multiprocessing
from data_util import build_speaker_dataset


In [2]:

temp_dir = tempfile.gettempdir()
# common_voice_path=os.path.abspath("../transaccent/data/en")
common_voice_path=os.path.abspath("/data")
audio_files_path = os.path.join(common_voice_path, "clips")
data_file_filename = "validated.tsv"
data_file_path = os.path.join(common_voice_path, data_file_filename)

# Retreive records
df = pd.read_csv(data_file_path, sep='\t', low_memory=False)
df = df[(df['down_votes'] < 1) & (df['gender'] == 'male') & (df['up_votes'] > 1)]
df.head()

,client_id,path,sentence,up_votes,down_votes,age,gender,accent
9,00c3f0e7c691ef30257d1bfa9adc410535b7ba3f48e344...,common_voice_en_18295850.mp3,The long-lived bridge still stands today.,2,0,twenties,male,NaN
70,05ba9bb1a4ac391849fa4461547967768f4d7df8ee52d7...,common_voice_en_19967535.mp3,The cemetery is now managed by three trusts.,2,0,fifties,male,african
81,06546553aed17027b4e638d4afb56f39b216026088cf40...,common_voice_en_17147389.mp3,Women form less than half of the group.,2,0,twenties,male,us
100,0838a82655be5a61349c2d2d86b60c22b5b84fea9826cb...,common_voice_en_18127728.mp3,Sunburn can be avoided by applying sunscreen o...,2,0,twenties,male,NaN
104,0899979e8d43a9faf448ddb5f4fc9a38a0fb4c120eaf34...,common_voice_en_17850951.mp3,Still waters run deep.,2,0,twenties,male,other


In [3]:
indian_df = build_speaker_dataset(df[(df['accent'] == 'indian') ], 1)
us_df = build_speaker_dataset(df[df['accent'] == 'us'], 1)
print(f"Retrieved {len(indian_df)} records with indian accents and {len(us_df)} records with us accents")


Retrieved 576 records with indian accents and 7827 records with us accents


In [41]:
# hop=192               #hop size (window size = 6*hop)
# sr=16000              #sampling rate
# min_level_db=-100     #reference values to normalize data
# ref_level_db=20

# shape=24              #length of time axis of split specrograms to feed to generator            
# vec_len=128           #length of vector generated by siamese vector
# # bs = 16               #batch size
# bs = 32               #batch size
# delta = 2.            #constant for siamese loss

In [53]:
from addict import Dict

def intialise_hparams(values):
    hparams = Dict(values)
    
    hparams.n_fft = 6 * hparams.hop # length of the windowed signal after padding with zeros
    hparams.win_length = 6 * hparams.hop
    hparams.max_spec_length = 10 * hparams.shape 

    return hparams

In [54]:
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

with open("hparams.yaml", "r") as fp:
    hparams_yaml = load(fp, Loader=Loader)

hparams = intialise_hparams(hparams_yaml)

print(hparams.n_fft)
print(hparams.trim_top_db)
print(hparams.win_length)
print(hparams.max_spec_length)
print(hparams.max_audio_samples)
print(hparams.generator.downscale.filter)

1152
40
1152
240
24064


256

In [7]:
def pad_audio_with_silence(wav, pad_value, hparams):
    """ We want to ensure that the size of all the audio samples are the same.
    If the audio file is smaller than the desired size then we will pad the ending with 0s
    If the audio file is larger than the desired size then we disregard the excess content
    """
    if len(wav) < hparams.max_audio_samples:
        pad_samples = hparams.max_audio_samples - len(wav)
        wav = np.pad(wav, (0, pad_samples), mode='constant', constant_values=pad_value)
    else: 
        wav = wav[:hparams.max_audio_samples]
    return wav

def trim_silence(wav, hparams):
    """ We want to ensure the beginning of the audio file is not silent"""
    return librosa.effects.trim(wav, top_db=hparams.trim_top_db, frame_length=hparams.trim_fft_size, hop_length=hparams.trim_hop_size)[0]

def load_audio(audio_file_name, hparams):
    """We use the librosa library to retrieve generate the representation of the audio file
    The sample rate used is specified as part of the hyperparameters.
    The function works for both mp3 and wav files
    """
    # The loading of mp3 files throw a warning, due to the volume of files we ignore this warning
    import warnings
    warnings.filterwarnings('ignore')
    y, _= librosa.load(audio_file_name, sr=hparams.sample_rate)
    return y

def convert_mp3_to_signal(audio_file_name, hparams):
    """Generate the signal representation of a single audio file"""
    y = load_audio(audio_file_name, hparams)
    if hparams.trim_silence:
        y = trim_silence(y, hparams)
    # pad to try to make all the audio files the same length
    y = pad_audio_with_silence(y, pad_value=0., hparams=hparams)
    return y

def process_audio(audio_path, hparams):
    """Helper function to generate the numpy array representation of the audio files"""
    return np.array(convert_mp3_to_signal(os.path.join(audio_files_path, audio_path), hparams))

def audio_array_for_accent(df, accent, hparams, limit_size=-1, tqdm=lambda x: x):

    import multiprocessing
    import numpy as np
    cpu_count = multiprocessing.cpu_count()
    
    input_mp3_files = df[df['accent'] == accent]['path'].to_numpy() 
    
    if limit_size > 0:
        print(f"Retrieved {len(input_mp3_files)} will only be using {limit_size}")
        input_mp3_files = input_mp3_files[:limit_size]
    
    executor = ProcessPoolExecutor(max_workers=cpu_count)
    futures = []
    index = 1
    
    for filepath in input_mp3_files:
        futures.append(executor.submit(partial(process_audio, filepath, hparams)))
        index += 1
    
    return np.array([future.result() for future in tqdm(futures) if future.result() is not None])



In [8]:
def melspecfunc(waveform, hparams):
    return librosa.feature.melspectrogram(y=waveform, 
                                          sr=hparams.sample_rate, 
                                          n_fft=hparams.n_fft, 
                                          win_length=hparams.win_length,
                                          hop_length=hparams.hop)

In [9]:
def normalize(S, hparams):
    return np.clip((((S - hparams.min_level_db) / -hparams.min_level_db)*2.)-1., -1, 1)

def prep(wv, hparams):
    S = np.array(melspecfunc(wv, hparams))
    S = librosa.power_to_db(S)-hparams.ref_level_db
    return normalize(S, hparams)

In [10]:
def process_wav(wav, hparams):
    S = np.array(prep(wav, hparams), dtype=np.float32)
    return np.expand_dims(S, -1)

def tospec(wvs, hparams):
    cpu_count = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(cpu_count)
    specs = pool.map(partial(process_wav, hparams=hparams), wvs)
    
    return specs

In [11]:
# split into equal chunk size
def split_spec(spec, chunk_size):
    return [spec[i * chunk_size: (i+1) * chunk_size] for i in range(int(np.ceil(len(spec) / chunk_size)))]

def splitcut(data, hparams):
    chunk_size = hparams.max_spec_length                                                             #max spectrogram length
    
    cpu_count = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(cpu_count)
    splits = pool.map(partial(split_spec,chunk_size=chunk_size), data)
    
    return np.array(splits)

In [29]:
def save_generated_representation(dir_name, spec, data):
    try:
        filename = f'{dir_name}/data.npz'
        if os.path.exists(filename):
            os.remove(filename)
        np.savez(filename, spec=spec, data=data)
        return True
    except:
        return False
    
def generate_representations(df, accent, output_dir, hparams):
    """Generate the representation of the audio files used by the DNNs"""
    # generate the signal representation for all the audio files restricted by the accent specified
    wv_all = audio_array_for_accent(df, accent, hparams, limit_size=len(indian_df))
    # Generate the the spectrogram representation
    spec = tospec(wv_all, hparams)
    # Split the spetrograms into equal size chucks
    data = splitcut(spec, hparams)
    # save the generated representations
    os.makedirs(indian_dir, exist_ok = True)
    save_generated_representation(output_dir, spec, data)
    return spec, data

In [30]:
indian_dir = '/data/wav/v3/indian'
os.makedirs(indian_dir, exist_ok = True)
us_dir = '/data/wav/v3/us'
os.makedirs(us_dir, exist_ok = True)

In [31]:
input_accent = 'indian'
target_accent = 'us'

In [32]:
aspec, adata = generate_representations(indian_df, input_accent, indian_dir, hparams)
bspec, adabdatata = generate_representations(us_df, target_accent, us_dir, hparams)

Retrieved 576 will only be using 576


In [ ]:
#Creating Tensorflow Datasets
import tensorflow as tf


@tf.function
def proc(x):
    return tf.image.random_crop(x, size=[hop, 3*shape, 1])

dsa = tf.data.Dataset.from_tensor_slices(adata)
    .repeat(hparams.num_dataset_repetitions)
    .map(proc, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .shuffle(hparams.shuffle_buffer_size)
    .batch(hparams.batch_size, drop_remainder=hparams.batch_drop_remainder)

dsb = tf.data.Dataset.from_tensor_slices(bdata)
    .repeat(hparams.num_dataset_repetitions)
    .map(proc, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .shuffle(hparams.shuffle_buffer_size)
    .batch(hparams.batch_size, drop_remainder=hparams.batch_drop_remainder)

In [35]:
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import sparse_ops
from tensorflow.python.ops import gen_math_ops
from tensorflow.python.ops import standard_ops
from tensorflow.python.eager import context
from tensorflow.python.framework import tensor_shape

In [ ]:
def l2normalize(v, eps=1e-12):
    return v / (tf.norm(v) + eps)

In [37]:

#Adding Spectral Normalization to convolutional layers
# each convolutional filter of both G and D is spectrally normalized as this greatly improves training stability - https://arxiv.org/abs/1802.05957

class ConvSN2D(tf.keras.layers.Conv2D):

    def __init__(self, filters, kernel_size, power_iterations=1, **kwargs):
        super(ConvSN2D, self).__init__(filters, kernel_size, **kwargs)
        self.power_iterations = power_iterations


    def build(self, input_shape):
        super(ConvSN2D, self).build(input_shape)

        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1

        self.u = self.add_weight(self.name + '_u',
            shape=tuple([1, self.kernel.shape.as_list()[-1]]), 
            initializer=tf.initializers.RandomNormal(0, 1),
            trainable=False
        )

    def compute_spectral_norm(self, W, new_u, W_shape):
        for _ in range(self.power_iterations):

            new_v = l2normalize(tf.matmul(new_u, tf.transpose(W)))
            new_u = l2normalize(tf.matmul(new_v, W))
            
        sigma = tf.matmul(tf.matmul(new_v, W), tf.transpose(new_u))
        W_bar = W/sigma

        with tf.control_dependencies([self.u.assign(new_u)]):
            W_bar = tf.reshape(W_bar, W_shape)

        return W_bar


    def call(self, inputs):
        W_shape = self.kernel.shape.as_list()
        W_reshaped = tf.reshape(self.kernel, (-1, W_shape[-1]))
        new_kernel = self.compute_spectral_norm(W_reshaped, self.u, W_shape)
        outputs = self._convolution_op(inputs, new_kernel)

        if self.use_bias:
            if self.data_format == 'channels_first':
                    outputs = tf.nn.bias_add(outputs, self.bias, data_format='NCHW')
            else:
                outputs = tf.nn.bias_add(outputs, self.bias, data_format='NHWC')
        if self.activation is not None:
            return self.activation(outputs)

        return outputs


In [38]:
class ConvSN2DTranspose(tf.keras.layers.Conv2DTranspose):

    def __init__(self, filters, kernel_size, power_iterations=1, **kwargs):
        super(ConvSN2DTranspose, self).__init__(filters, kernel_size, **kwargs)
        self.power_iterations = power_iterations


    def build(self, input_shape):
        super(ConvSN2DTranspose, self).build(input_shape)

        if self.data_format == 'channels_first':
            channel_axis = 1
        else:
            channel_axis = -1

        self.u = self.add_weight(self.name + '_u',
            shape=tuple([1, self.kernel.shape.as_list()[-1]]), 
            initializer=tf.initializers.RandomNormal(0, 1),
            trainable=False
        )

    def compute_spectral_norm(self, W, new_u, W_shape):
        for _ in range(self.power_iterations):

            new_v = l2normalize(tf.matmul(new_u, tf.transpose(W)))
            new_u = l2normalize(tf.matmul(new_v, W))
            
        sigma = tf.matmul(tf.matmul(new_v, W), tf.transpose(new_u))
        W_bar = W/sigma

        with tf.control_dependencies([self.u.assign(new_u)]):
            W_bar = tf.reshape(W_bar, W_shape)

        return W_bar

    def call(self, inputs):
        W_shape = self.kernel.shape.as_list()
        W_reshaped = tf.reshape(self.kernel, (-1, W_shape[-1]))
        new_kernel = self.compute_spectral_norm(W_reshaped, self.u, W_shape)

        inputs_shape = array_ops.shape(inputs)
        batch_size = inputs_shape[0]
        if self.data_format == 'channels_first':
            h_axis, w_axis = 2, 3
        else:
            h_axis, w_axis = 1, 2

        height, width = inputs_shape[h_axis], inputs_shape[w_axis]
        kernel_h, kernel_w = self.kernel_size
        stride_h, stride_w = self.strides

        if self.output_padding is None:
            out_pad_h = out_pad_w = None
        else:
            out_pad_h, out_pad_w = self.output_padding

        out_height = conv_utils.deconv_output_length(height,
                                                    kernel_h,
                                                    padding=self.padding,
                                                    output_padding=out_pad_h,
                                                    stride=stride_h,
                                                    dilation=self.dilation_rate[0])
        out_width = conv_utils.deconv_output_length(width,
                                                    kernel_w,
                                                    padding=self.padding,
                                                    output_padding=out_pad_w,
                                                    stride=stride_w,
                                                    dilation=self.dilation_rate[1])
        if self.data_format == 'channels_first':
            output_shape = (batch_size, self.filters, out_height, out_width)
        else:
            output_shape = (batch_size, out_height, out_width, self.filters)

        output_shape_tensor = array_ops.stack(output_shape)
        outputs = K.conv2d_transpose(
            inputs,
            new_kernel,
            output_shape_tensor,
            strides=self.strides,
            padding=self.padding,
            data_format=self.data_format,
            dilation_rate=self.dilation_rate)

        if not context.executing_eagerly():
            out_shape = self.compute_output_shape(inputs.shape)
            outputs.set_shape(out_shape)

        if self.use_bias:
            outputs = tf.nn.bias_add(
              outputs,
              self.bias,
              data_format=conv_utils.convert_data_format(self.data_format, ndim=4))

        if self.activation is not None:
            return self.activation(outputs)
        return outputs  

In [40]:
class DenseSN(tf.keras.layers.Dense):
    def build(self, input_shape):
        super(DenseSN, self).build(input_shape)

        self.u = self.add_weight(self.name + '_u',
            shape=tuple([1, self.kernel.shape.as_list()[-1]]), 
            initializer=tf.initializers.RandomNormal(0, 1),
            trainable=False)
        
    def compute_spectral_norm(self, W, new_u, W_shape):
        new_v = l2normalize(tf.matmul(new_u, tf.transpose(W)))
        new_u = l2normalize(tf.matmul(new_v, W))
        sigma = tf.matmul(tf.matmul(new_v, W), tf.transpose(new_u))
        W_bar = W/sigma
        with tf.control_dependencies([self.u.assign(new_u)]):
            W_bar = tf.reshape(W_bar, W_shape)
        return W_bar
        
    def call(self, inputs):
        W_shape = self.kernel.shape.as_list()
        W_reshaped = tf.reshape(self.kernel, (-1, W_shape[-1]))
        new_kernel = self.compute_spectral_norm(W_reshaped, self.u, W_shape)
        rank = len(inputs.shape)
        if rank > 2:
            outputs = standard_ops.tensordot(inputs, new_kernel, [[rank - 1], [0]])
            if not context.executing_eagerly():
                shape = inputs.shape.as_list()
                output_shape = shape[:-1] + [self.units]
                outputs.set_shape(output_shape)
        else:
            inputs = math_ops.cast(inputs, self._compute_dtype)
            if K.is_sparse(inputs):
                outputs = sparse_ops.sparse_tensor_dense_matmul(inputs, new_kernel)
            else:
                outputs = gen_math_ops.mat_mul(inputs, new_kernel)
        if self.use_bias:
            outputs = tf.nn.bias_add(outputs, self.bias)
        if self.activation is not None:
            return self.activation(outputs)
        return outputs


In [41]:
#Networks Architecture


def conv2d(layer_input, 
           filters, 
           kernel_size=4, 
           strides=2, 
           padding='same', 
           leaky=True, 
           bnorm=True, 
           sn=True, 
           init_fun=tf.keras.initializers.he_uniform()):
    
    if leaky:
        Activ = LeakyReLU(alpha=0.2)
    else:
        Activ = ReLU()
    if sn:
        d = ConvSN2D(
            filters, 
            kernel_size=kernel_size, 
            strides=strides, 
            padding=padding, 
            kernel_initializer=init_fun, 
            use_bias=False)(layer_input)
    else:
        d = Conv2D(
            filters, 
            kernel_size=kernel_size, 
            strides=strides, 
            padding=padding, 
            kernel_initializer=init_fun, 
            use_bias=False)(layer_input)
    
    if bnorm:
        d = BatchNormalization()(d)

    d = Activ(d)
    return d

def deconv2d(layer_input, 
             layer_res, 
             filters, 
             kernel_size=4, 
             conc=True, 
             scalev=False, 
             bnorm=True, 
             up=True, 
             padding='same', 
             strides=2, 
             init_fun=tf.keras.initializers.he_uniform()):
    if up:
        u = UpSampling2D((1,2))(layer_input)
        u = ConvSN2D(
            filters, 
            kernel_size, 
            strides=(1,1), 
            kernel_initializer=init_fun, 
            use_bias=False, 
            padding=padding)(u)
    else:
        u = ConvSN2DTranspose(
            filters, 
            kernel_size, 
            strides=strides, 
            kernel_initializer=init_fun, 
            use_bias=False, 
            padding=padding)(layer_input)
    
    if bnorm:
        u = BatchNormalization()(u)
    u = LeakyReLU(alpha=0.2)(u)
    if conc:
        u = Concatenate()([u,layer_res])
    return u

#Extract function: splitting spectrograms
def extract_image(im, hparams):
    im1 = Cropping2D(((0,0), (0, 2*(im.shape[2]//3))))(im)
    im2 = Cropping2D(((0,0), (im.shape[2]//3,im.shape[2]//3)))(im)
    im3 = Cropping2D(((0,0), (2*(im.shape[2]//3), 0)))(im)
    return im1,im2,im3

#Assemble function: concatenating spectrograms
def assemble_image(lsim, hparams):
    im1,im2,im3 = lsim
    imh = Concatenate(2)([im1,im2,im3])
    return imh

#U-NET style architecture
def build_generator(input_shape, hparams):
    h,w,c = input_shape
    inp = Input(shape=input_shape)
    
    #downscaling
    g0 = tf.keras.layers.ZeroPadding2D((0,1))(inp)
    g1 = conv2d(g0, filters=hparams.generator.downscale.filter, kernel_size=(h,3), strides=1, padding='valid')
    g2 = conv2d(g1, filters=hparams.generator.downscale.filter, kernel_size=(1,9), strides=(1,2))
    g3 = conv2d(g2, filters=hparams.generator.downscale.filter, kernel_size=(1,7), strides=(1,2))
    
    #upscaling
    g4 = deconv2d(g3, layer_res=g2, filters=hparams.generator.upscale.filter, kernel_size=(1,7), strides=(1,2))
    g5 = deconv2d(g4, layer_res=g1, filters=hparams.generator.upscale.filter, kernel_size=(1,9), strides=(1,2), bnorm=False)
    g6 = ConvSN2DTranspose(1, 
                           kernel_size=(h,1), 
                           strides=(1,1), 
                           kernel_initializer=tf.keras.initializers.he_uniform(), 
                           padding='valid', 
                           activation='tanh')(g5)
    
    return Model(inp,g6, name='G')

#Siamese Network
def build_siamese(input_shape, hparams):
    h,w,c = input_shape
    inp = Input(shape=input_shape)
    g1 = conv2d(inp, filters=hparams.siamese.filter, kernel_size=(h,3), strides=1, padding='valid', sn=False)
    g2 = conv2d(g1, filters=hparams.siamese.filter, kernel_size=(1,9), strides=(1,2), sn=False)
    g3 = conv2d(g2, filters=hparams.siamese.filter, kernel_size=(1,7), strides=(1,2), sn=False)
    g4 = Flatten()(g3)
    g5 = Dense(hparams.vec_len)(g4)
    return Model(inp, g5, name='S')

#Discriminator (Critic) Network
def build_critic(input_shape, hparams):
    h,w,c = input_shape
    inp = Input(shape=input_shape)
    g1 = conv2d(inp, filters=hparams.discriminator.filter, kernel_size=(h,3), strides=1, padding='valid', bnorm=False)
    g2 = conv2d(g1, filters=hparams.discriminator.filter, kernel_size=(1,9), strides=(1,2), bnorm=False)
    g3 = conv2d(g2, filters=hparams.discriminator.filter, kernel_size=(1,7), strides=(1,2), bnorm=False)
    g4 = Flatten()(g3)
    g4 = DenseSN(1, kernel_initializer=tf.keras.initializers.he_uniform())(g4)
    return Model(inp, g4, name='C')

In [ ]:
#Load past models from path to resume training or test
def load(path, hparams):
    gen = build_generator((hparams.hop, hparams.shape,1), hparams)
    siam = build_siamese((hparams.hop, hparams.shape,1), hparams)
    critic = build_critic((hparams.hop, 3 * hparams.shape,1), hparams)
    
    gen.load_weights(path+'/gen.h5')
    critic.load_weights(path+'/critic.h5')
    siam.load_weights(path+'/siam.h5')
    return gen,critic,siam

#Build models
def build(hparams):
    gen = build_generator((hparams.hop, hparams.shape,1), hparams)
    siam = build_siamese((hparams.hop, hparams.shape,1), hparams)
    critic = build_critic((hparams.hop, 3 * hparams.shape,1), hparams) #the discriminator accepts as input spectrograms of triple the width of those generated by the generator
    return gen,critic,siam

#Generate a random batch to display current training results
def testgena(aspec, hparams):
    sw = True
    while sw:
        a = np.random.choice(aspec)
        if a.shape[1]//hparams.shape != 1:
            sw=False
    
    dsa = []
    
    if a.shape[1]//hparams.shape > 6:
        num=6
    else:
        num=a.shape[1]//hparams.shape

    rn = np.random.randint(a.shape[1]-(num * hparams.shape))
    
    for i in range(num):
        im = a[ : , rn + (i * hparams.shape) : rn+(i * hparams.shape) + hparams.shape]
        im = np.reshape(im, (im.shape[0], im.shape[1],1))
        dsa.append(im)
        
    return np.array(dsa, dtype=np.float32)

#Show results mid-training
def save_test_image_full(path, aspec, gen, hparams):
    a = testgena(aspec, hparams)

    ab = gen(a, training=False)
    ab = testass(ab)
    a = testass(a)
    
    abwv = deprep(ab)
    awv = deprep(a)
    sf.write(path+'/new_file.wav', abwv, sr)
    IPython.display.display(IPython.display.Audio(np.squeeze(abwv), rate=sr))
    IPython.display.display(IPython.display.Audio(np.squeeze(awv), rate=sr))
    fig, axs = plt.subplots(ncols=2)
    axs[0].imshow(np.flip(a, -2), cmap=None)
    axs[0].axis('off')
    axs[0].set_title('Source')
    axs[1].imshow(np.flip(ab, -2), cmap=None)
    axs[1].axis('off')
    axs[1].set_title('Generated')
    plt.show()

#Save in training loop
def save_end(epoch,gloss,closs,mloss,n_save=3,save_path='/data/output/models'):                 #use custom save_path (i.e. Drive '../content/drive/My Drive/')
    if epoch % n_save == 0:
        print('Saving...')
        path = f'{save_path}/MELGANVC-{str(gloss)[:9]}-{str(closs)[:9]}-{str(mloss)[:9]}'
        os.mkdir(path)
        gen.save_weights(path+'/gen.h5')
        critic.save_weights(path+'/critic.h5')
        siam.save_weights(path+'/siam.h5')
        save_test_image_full(path)